# Implementation of Modern Portfolio Theory

In [2]:
# import libraries
import numpy as np
import pandas as pd
import datetime as dt
import yfinance as yf
import scipy

# Modern Portfolio Theory (Markowitz Efficient Frontier)

Efficient Frontier: This represents the set of optimal portfolios offering the highest expected return for a defined level of risk. Portfolios on the efficient frontier are considered well-diversified and optimal.

### Modern Portfolio Theory Formula

For a given **risk tolerance** $q \geq 0 $, the efficient frontier is determined by minimizing:

$$
w^T \Sigma w - q R^T w
$$

where:

- $w$: Vector of portfolio weights, summing to 1 (weights can be negative).
- $\Sigma$: Covariance matrix of asset returns.
- $q$: Risk tolerance parameter; $q = 0$: Minimum variance portfolio. $q \to \infty$: Portfolios with unbounded expected return and risk.
- $R$: Vector of expected asset returns.
- $w^T \Sigma w$: Portfolio return variance.
- $R^T w$: Expected portfolio return.

In [3]:
# Import data
def getData(stocks, start, end):
    stockData = yf.download(stocks, start=start, end=end)['Close']

    # Calculate daily returns, mean returns, and covariance matrix
    returns = stockData.pct_change()
    meanReturns = returns.mean()
    covMatrix = returns.cov()
    return meanReturns, covMatrix

def portfolioPerformance(weights, meanReturns, covMatrix):
    weights = np.array(weights)  # Convert weights to NumPy array
    returns = np.sum(meanReturns * weights) * 252  # Annualized return
    std = np.sqrt(np.dot(weights.T, np.dot(covMatrix, weights))) * np.sqrt(252)  # Annualized standard deviation
    return returns, std

# Define stock tickers
stockList = ['AAPL', 'MSFT', 'TSLA']

# Define date range
endDate = dt.datetime.now()
startDate = endDate - dt.timedelta(days=365)

# Portfolio weights
weights = [0.3, 0.3, 0.4]

# Fetch mean returns and covariance matrix
meanReturns, covMatrix = getData(stockList, start=startDate.strftime('%Y-%m-%d'), end=endDate.strftime('%Y-%m-%d'))

# Calculate portfolio performance
returns, std = portfolioPerformance(weights, meanReturns, covMatrix)
print(f"Portfolio Return: {round(returns * 100, 2)}%")
print(f"Portfolio Standard Deviation: {round(std * 100, 2)}%")

[*********************100%***********************]  3 of 3 completed

1 Failed download:
['TSLA']: OperationalError('database is locked')


Portfolio Return: 10.53%
Portfolio Standard Deviation: nan%


/var/folders/cd/7ncxf4rn7q19v1nd6sf6h8y40000gn/T/ipykernel_5400/2333569247.py:6: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  returns = stockData.pct_change()


# Portfolio Optimization: Minimizing Risk

We can formulate this portfolio optimization problem in three equivalent ways. One of them is:

### Minimize Portfolio Risk
Minimize the portfolio risk subject to a constraint expressing a lower bound on the portfolio return:

### Problem Formulation
$$
\min \ \frac{1}{2} \mathbf{w}^T \Sigma \mathbf{w}
$$

**Subject to:**

1. $ \mathbf{m}^T \mathbf{w} \geq \mu_b $ (Portfolio Return above $ \mu_b $)  
2. $ \mathbf{e}^T \mathbf{w} = 1 $ 

### Variables:
- $ \mathbf{w} $: Vector of portfolio weightings  
- $ \Sigma $: Covariance matrix  
- $ \mathbf{m} $: Vector of asset returns  
- $ \mathbf{e} $: Vector of 1s  

This is what we want to use in order to generate the efficient frontier later on.

We're just going to be focusing on a small subset which is getting the weights of the maximum sharp ratio portfolio.

In [4]:
# Fetch stock data
def getData(stocks, start, end):
    stockData = yf.download(stocks, start=start, end=end)['Close']
    returns = stockData.pct_change()
    meanReturns = returns.mean()
    covMatrix = returns.cov()
    return meanReturns, covMatrix


# Calculate portfolio performance
def portfolioPerformance(weights, meanReturns, covMatrix):
    weights = np.array(weights)
    returns = np.sum(meanReturns * weights) * 252
    std = np.sqrt(np.dot(weights.T, np.dot(covMatrix, weights))) * np.sqrt(252)
    return returns, std

# Negative Sharpe Ratio
def negativeSR(weights, meanReturns, covMatrix, riskFreeRate=0):
    pReturns, pStd = portfolioPerformance(weights, meanReturns, covMatrix)
    # Add small constant to avoid division by zero
    return -(pReturns - riskFreeRate) / (pStd + 1e-6)

# Maximize Sharpe Ratio
def maxSR(meanReturns, covMatrix, riskFreeRate=0, constrainSet=(0, 1)):
    """
    Minimize the negative Sharpe Ratio by altering the portfolio weights
    """
    numAssets = len(meanReturns)
    args = (meanReturns, covMatrix, riskFreeRate)
    constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
    bounds = tuple(constrainSet for asset in range(numAssets))
    result = scipy.optimize.minimize(  # Use minimize directly
        negativeSR,
        numAssets * [1. / numAssets],
        args=args,
        method='SLSQP',
        bounds=bounds,
        constraints=constraints
    )
    return result

# Portfolio Variance
def portfolioVariance(weights, meanReturns, covMatrix):
    weights = np.array(weights)
    variance = np.dot(weights.T, np.dot(covMatrix, weights))
    return variance

# Minimize Portfolio Variance
def minimizeVariance(meanReturns, covMatrix, constrainSet=(0, 1)):
    """
    Minimize the portfolio variance by altering the weights/allocation of assets in the portfolio
    """
    numAssets = len(meanReturns)
    args = (meanReturns, covMatrix)
    constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
    bounds = tuple(constrainSet for asset in range(numAssets))
    result = scipy.optimize.minimize(  # Use minimize directly
        portfolioVariance,
        numAssets * [1. / numAssets],
        args=args,
        method='SLSQP',
        bounds=bounds,
        constraints=constraints
    )
    return result

# Main
stockList = ['AAPL', 'MSFT', 'TSLA']

endDate = dt.datetime.now()
startDate = endDate - dt.timedelta(days=365)

# Fetch data and calculate optimal portfolio
meanReturns, covMatrix = getData(stockList, start=startDate.strftime('%Y-%m-%d'), end=endDate.strftime('%Y-%m-%d'))

# Maximize Sharpe Ratio
result = maxSR(meanReturns, covMatrix)
maxSR, maxWeights = -result['fun'], result['x']  # Sharpe Ratio is the negative of the minimized value
print("Maximum Sharpe Ratio:", maxSR)
print("Optimal Weights (Max Sharpe Ratio):", maxWeights)

# Minimize Variance
minVarResult = minimizeVariance(meanReturns, covMatrix)
minVar, minVarWeights = minVarResult['fun'], minVarResult['x']
print("Minimum Variance:", minVar)
print("Optimal Weights (Min Variance):", minVarWeights)


[                       0%                       ]

[*********************100%***********************]  3 of 3 completed


Maximum Sharpe Ratio: 1.184980372547164
Optimal Weights (Max Sharpe Ratio): [0.69494909 0.10446686 0.20058405]
Minimum Variance: 0.00029003880924571797
Optimal Weights (Min Variance): [0.33333333 0.33333333 0.33333333]


In [5]:
# Fetch stock data
def getData(stocks, start, end):
    stockData = yf.download(stocks, start=start, end=end)['Close']
    returns = stockData.pct_change()
    meanReturns = returns.mean()
    covMatrix = returns.cov()
    return meanReturns, covMatrix


# Calculate portfolio performance
def portfolioPerformance(weights, meanReturns, covMatrix):
    weights = np.array(weights)
    returns = np.sum(meanReturns * weights) * 252
    std = np.sqrt(np.dot(weights.T, np.dot(covMatrix, weights))) * np.sqrt(252)
    return returns, std

# Negative Sharpe Ratio
def negativeSR(weights, meanReturns, covMatrix, riskFreeRate=0):
    pReturns, pStd = portfolioPerformance(weights, meanReturns, covMatrix)
    # Add small constant to avoid division by zero
    return -(pReturns - riskFreeRate) / (pStd + 1e-6)

# Maximize Sharpe Ratio
def maxSR(meanReturns, covMatrix, riskFreeRate=0, constrainSet=(0, 1)):
    """
    Minimize the negative Sharpe Ratio by altering the portfolio weights
    """
    numAssets = len(meanReturns)
    args = (meanReturns, covMatrix, riskFreeRate)
    constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
    bounds = tuple(constrainSet for asset in range(numAssets))
    result = scipy.optimize.minimize(  # Use minimize directly
        negativeSR,
        numAssets * [1. / numAssets],
        args=args,
        method='SLSQP',
        bounds=bounds,
        constraints=constraints
    )
    return result

# Portfolio Variance
def portfolioVariance(weights, meanReturns, covMatrix):
    weights = np.array(weights)
    variance = np.dot(weights.T, np.dot(covMatrix, weights))
    return variance

# Minimize Portfolio Variance
def minimizeVariance(meanReturns, covMatrix, constrainSet=(0, 1)):
    """
    Minimize the portfolio variance by altering the weights/allocation of assets in the portfolio
    """
    numAssets = len(meanReturns)
    args = (meanReturns, covMatrix)
    constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
    bounds = tuple(constrainSet for asset in range(numAssets))
    result = scipy.optimize.minimize(  # Use minimize directly
        portfolioVariance,
        numAssets * [1. / numAssets],
        args=args,
        method='SLSQP',
        bounds=bounds,
        constraints=constraints
    )
    return result

# Main
stockList = ['AAPL', 'MSFT', 'TSLA']

endDate = dt.datetime.now()
startDate = endDate - dt.timedelta(days=365)

# Fetch data and calculate optimal portfolio
meanReturns, covMatrix = getData(stockList, start=startDate.strftime('%Y-%m-%d'), end=endDate.strftime('%Y-%m-%d'))

def calculatedResults(meanReturns, covMatrix, riskFreeRate=0, constraintSet=(0, 1)):
    """
    Read in mean, cov matrix, and other financial information
    Output, Max SR, Min Volatility, efficient frontier
    """
    # Max Sharp Ratio Portfolio
    maxSR_Portfolio = maxSR(meanReturns, covMatrix)
    maxSR_returns, maxSR_std = portfolioPerformance(maxSR_Portfolio['x'], meanReturns, covMatrix)
    maxSR_returns, maxSR_std = round(maxSR_returns * 100, 2), round(maxSR_std * 100, 2)
    maxSR_allocation = pd.DataFrame(maxSR_Portfolio['x'], index=meanReturns.index, columns=['allocation'])
    maxSR_allocation.allocation = [round(i * 100, 0) for i in maxSR_allocation.allocation]

    # Min Volatility Portfolio
    minVol_Portfolio = minimizeVariance(meanReturns, covMatrix)
    minVol_returns, minVol_std = portfolioPerformance(minVol_Portfolio['x'], meanReturns, covMatrix)
    minVol_returns, minVol_std = round(minVol_returns * 100, 2), round(minVol_std * 100, 2)
    minVol_allocation = pd.DataFrame(minVol_Portfolio['x'], index=meanReturns.index, columns=['allocation'])
    minVol_allocation.allocation = [round(i * 100, 0) for i in minVol_allocation.allocation]
    return maxSR_returns, maxSR_std, maxSR_allocation, minVol_returns, minVol_std, minVol_allocation

print(calculatedResults(meanReturns, covMatrix))

[                       0%                       ]

[*********************100%***********************]  3 of 3 completed

(np.float64(28.25), np.float64(23.84),         allocation
Ticker            
AAPL          69.0
MSFT          10.0
TSLA          20.0, np.float64(29.92), np.float64(27.04),         allocation
Ticker            
AAPL          33.0
MSFT          33.0
TSLA          33.0)


In [7]:
def portfolioReturn(weights, meanReturns, covMatrix):
    return portfolioPerformance(weights, meanReturns, covMatrix)[0]
def efficientOpt(meanReturns, covMatrix, returnTarget, constraintSet=(0, 1)):
    """
    For each return Target, we want to optimize the portfolio for min variance
    """
    numAssets = len(meanReturns)
    args = (meanReturns, covMatrix)

    constraints = ({'type': 'eq', 'fun': lambda x: portfolioReturn(x, meanReturns, covMatrix) - returnTarget},
                   {'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
    bound = constraintSet
    bounds = (constraintSet for asset in range(numAssets))
    effOpt = scipy.optimize.minimize(portfolioVariance, numAssets * [1. / numAssets], args=args, method='SLSQP',
                                     bounds = bounds, constraints = constraints)
    return effOpt

def calculatedResults(meanReturns, covMatrix, riskFreeRate=0, constraintSet=(0, 1)):
    """
    Read in mean, cov matrix, and other financial information
    Output, Max SR, Min Volatility, efficient frontier
    """
    # Max Sharp Ratio Portfolio
    maxSR_Portfolio = maxSR(meanReturns, covMatrix)
    maxSR_returns, maxSR_std = portfolioPerformance(maxSR_Portfolio['x'], meanReturns, covMatrix)
    maxSR_returns, maxSR_std = round(maxSR_returns * 100, 2), round(maxSR_std * 100, 2)
    maxSR_allocation = pd.DataFrame(maxSR_Portfolio['x'], index=meanReturns.index, columns=['allocation'])
    maxSR_allocation.allocation = [round(i * 100, 0) for i in maxSR_allocation.allocation]
    
    # Min Volatility Portfolio
    minVol_Portfolio = minimizeVariance(meanReturns, covMatrix)
    minVol_returns, minVol_std = portfolioPerformance(minVol_Portfolio['x'], meanReturns, covMatrix)
    minVol_returns, minVol_std = round(minVol_returns * 100, 2), round(minVol_std * 100, 2)
    minVol_allocation = pd.DataFrame(minVol_Portfolio['x'], index=meanReturns.index, columns=['allocation'])
    minVol_allocation.allocation = [round(i * 100, 0) for i in minVol_allocation.allocation]

    # Efficient Frontier
    efficientList = []
    targetReturns = np.linspace(minVol_returns, maxSR_returns, 20)
    for target in targetReturns:
        efficientList.append(efficientOpt(meanReturns, covMatrix, target)['fun'])

    return maxSR_returns, maxSR_std, maxSR_allocation, minVol_returns, minVol_std, minVol_allocation

print(calculatedResults(meanReturns, covMatrix))

(np.float64(28.25), np.float64(23.84),         allocation
Ticker            
AAPL          69.0
MSFT          10.0
TSLA          20.0, np.float64(29.92), np.float64(27.04),         allocation
Ticker            
AAPL          33.0
MSFT          33.0
TSLA          33.0)


In [14]:
# Plot
!pip install plotly

In [15]:
import plotly.graph_objects as go

In [ ]:
def calculatedResults(meanReturns, covMatrix, riskFreeRate=0, constraintSet=(0, 1)):
    """
    Read in mean, cov matrix, and other financial information
    Output, Max SR, Min Volatility, efficient frontier
    """
    # Max Sharp Ratio Portfolio
    maxSR_Portfolio = maxSR(meanReturns, covMatrix)
    maxSR_returns, maxSR_std = portfolioPerformance(maxSR_Portfolio['x'], meanReturns, covMatrix)
    maxSR_allocation = pd.DataFrame(maxSR_Portfolio['x'], index=meanReturns.index, columns=['allocation'])
    maxSR_allocation.allocation = [round(i * 100, 0) for i in maxSR_allocation.allocation]
    
    # Min Volatility Portfolio
    minVol_Portfolio = minimizeVariance(meanReturns, covMatrix)
    minVol_returns, minVol_std = portfolioPerformance(minVol_Portfolio['x'], meanReturns, covMatrix)
    minVol_allocation = pd.DataFrame(minVol_Portfolio['x'], index=meanReturns.index, columns=['allocation'])
    minVol_allocation.allocation = [round(i * 100, 0) for i in minVol_allocation.allocation]

    # Efficient Frontier
    efficientList = []
    targetReturns = np.linspace(minVol_returns, maxSR_returns, 20)
    for target in targetReturns:
        efficientList.append(efficientOpt(meanReturns, covMatrix, target)['fun'])

    maxSR_returns, maxSR_std = round(maxSR_returns * 100, 2), round(maxSR_std * 100, 2)
    minVol_returns, minVol_std = round(minVol_returns * 100, 2), round(minVol_std * 100, 2)

    return maxSR_returns, maxSR_std, maxSR_allocation, minVol_returns, minVol_std, minVol_allocation, efficientList, targetReturns

def EF_graph(meanReturns, covMatrix, riskFreeRate=0, constraintSet=(0, 1)):
    """
    Return a graph ploting the min vol, max sr and efficient frontier
    """
    maxSR_returns, maxSR_std, maxSR_allocation, minVol_returns, minVol_std, minVol_allocation, efficientList, targetReturns = calculatedResults(meanReturns, covMatrix)
    
    # Max SR
    maxSharpRatio = go.Scatter(
        name='Maximum Sharp Ratio',
        mode='markers',
        x=[maxSR_std],
        y=[maxSR_returns],
        marker=dict(
            color='red',
            size=15,
            line=dict(
                color='black',
                width=3
            )
        )
    )

    # Min Volatility
    minVol = go.Scatter(
        name='Minimum Volatility',
        mode='markers',
        x=[minVol_std],
        y=[minVol_returns],
        marker=dict(
            color='green',
            size=15,
            line=dict(
                color='black',
                width=3
            )
        )
    )

    # Efficient Frontier
    EF_curve = go.Scatter(
        name='Efficient Frontier',
        mode='lines',
        x=[round(ef_std * 100, 2) for ef_std in efficientList],
        y=[round(target * 100, 2) for target in targetReturns],
        line=dict(
            color='black',
            width=4,
            dash='dashdot'
        )
    )
    
    data = [maxSharpRatio, minVol, EF_curve]

    layout = go.Layout(
        title='Portfolio Optimization with the Efficient Frontier',
        yaxis = dict(title = 'Annualized Return (%)'),
        xaxis = dict(title = 'Annualized Volatility (%)'),
        showlegend = True,
        legend = dict(
            x = 0.75, y = 0, traceorder = 'normal',
            bgcolor = '#E2E2E2',
            bordercolor = 'black',
            borderwidth = 2),
        width = 800,
        height = 600    
    )
    
    fig = go.Figure(data=data, layout=layout)
    return fig.show()

EF_graph(meanReturns, covMatrix)